In [ ]:
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq, scaler=None):
    # 모델을 학습 모드로 설정합니다.
    model.train()
    
    # 메트릭 로거를 초기화합니다.
    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter("lr", utils.SmoothedValue(window_size=1, fmt="{value:.6f}"))
    header = f"Epoch: [{epoch}]"
    
    # 첫 번째 에포크일 경우, 선형 학습률 스케줄러를 초기화합니다.
    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1.0 / 1000
        warmup_iters = min(1000, len(data_loader) - 1)
        
        lr_scheduler = torch.optim.lr_scheduler.LinearLR(
            optimizer, start_factor=warmup_factor, total_iters=warmup_iters
        )

    # 데이터로더를 통해 반복하면서 학습을 수행합니다.
    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        # 이미지를 GPU로 이동합니다.
        images = [image.to(device) for image in images]

        # Mixed precision 사용 여부에 따라 자동형변환을 활성화하고 손실을 계산합니다.
        with torch.cuda.amp.autocast(enabled=scaler is not None):
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

        # 모든 GPU에 대한 손실을 줄입니다.
        loss_dict_reduced = utils.reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())
        
        # 손실 값이 유효한지 확인합니다.
        loss_value = losses_reduced.item()
        if not math.isfinite(loss_value):
            print(f"Loss is {loss_value}, stopping training")
            print(loss_dict_reduced)
            sys.exit(1)

        # 그래디언트를 초기화하고 역전파를 수행합니다.
        optimizer.zero_grad()
        if scaler is not None:
            scaler.scale(losses).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            losses.backward()
            optimizer.step()

        # 학습률 스케줄러를 업데이트합니다.
        if lr_scheduler is not None:
            lr_scheduler.step()

        # 메트릭 로거를 업데이트합니다.
        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])
        
    return metric_logger
